In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [2]:

def get_cve_details(cve_id):
    print("DEBUGGING -- cve_id:" + cve_id)
    # Set the headers
    #headers = {
    #    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    #}
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                             '(KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
    # Construct the URL
    url = f'https://www.cvedetails.com/cve/{cve_id}'

    # Send the GET request to the URL
    response = requests.get(url, headers=headers, verify=False)

    # Check the response status code
    if response.status_code != 200:
        raise Exception(f'Error: Invalid CVE ID "{cve_id}".')

    # Parse the HTML response
    soup = BeautifulSoup(response.text, 'html.parser')
    #print("DEBUGGING -- response.text - "+response.text)
    # Extract the CVE ID
    #cve_id = cve_id
    #print("DEBUGGING -- cve_id:"+cve_id)
    cve_content_div = soup.find('div', attrs={'id': 'contentdiv'})
    #the cve has no entry in cvedetails yet
    #alert= cve_content_div.find('div', class_='alert alert-secondary my-4').text.strip()
    if  cve_content_div.find('div', class_='alert alert-secondary my-4'):
        message = f'>> Failed to parse HTML elements cve_content_div << \n\n'
        print(f'ERROR {cve_id}{message} ')
        return {
            'cve_id': cve_id
        }
        ## Send the result to webhook. ##
        #sys.exit(0)
        return
    #the cve has no entry in cvedetails yet
 
   

    #cve_id = cve_content_div.find('h1').find('a').text.strip()
    description = cve_content_div.find('div', class_='cvedetailssummary-text').text.strip()
    #print("DEBUGGING -- description:" + description)
    published_updated_elements = cve_content_div.find_all('div', class_='d-inline-block')
    published_date = published_updated_elements[0].text.strip().replace("Published", "").strip()
    updated_date = published_updated_elements[1].text.strip().replace("Updated", "").strip()
    #print("DEBUGGING -- published_date:" + published_date)
    #print("DEBUGGING -- updated_date:" + updated_date)

    # Extract the Vulnerability category
    vuln_cat_elements = cve_content_div.find_all('span', class_='ssc-vuln-cat')
    total_vuln_cat=[]
    for x in range(len(vuln_cat_elements )):
        total_vuln_cat+= vuln_cat_elements[x]
    #print("DEBUGGING -- vuln_cat:", *total_vuln_cat, sep=",")

    # Extract the base score
    cve_content_div2 = soup.find('table',  class_='table table-borderless')
    if  not cve_content_div2:
        message = f'>> Failed to parse HTML elements cve_content_div2 << \n\n'
        print(f'ERROR {cve_id}{message} ')
        return {
            'cve_id': cve_id
        }
    base_score_elements = cve_content_div2.find_all('td', class_='ps-2')
    base_score = base_score_elements[0].find('div', class_='cvssbox').text.strip()
    base_severity = base_score_elements[1].text.strip()
    vector = base_score_elements[2].text.strip()
    exploitability = base_score_elements[3].text.strip()
    impact = base_score_elements[4].text.strip()

    #
    #cwe_heading = cve_content_div.find('h2', string='CWE ids for ' + cve_id)
    #if cwe_heading:
    #    cwe_item = cwe_heading.find_next('a')
    #    if cwe_item:
    #        cwe_id = cwe_item.text.strip()
    #else:
    #    cwe_id = f'Not found CWE ids for {cve_id}'

    # Extract the published date
    #published_date = soup.find('div', class_='d-inline-block').text.strip().replace("Published", "").strip()
    #print("DEBUGGING -- published_date:" + published_date)
    # Extract the CWE ID
    cwe_id_element = soup.find('h2', string='CWE ids for ' + cve_id)
    if cwe_id_element:
        cwe_item = cwe_id_element.find_next('a')
        if cwe_item:
            cwe_id = cwe_item.text.strip()
    else:
        cwe_id = 'Not found CWE ids for ' + cve_id
    #print("DEBUGGING -- cwe_id :" + cwe_id )

    references_heading = cve_content_div.find('h2', string='References for ' + cve_id)
    if references_heading:
        references_list = references_heading.find_next('ul', class_='list-group')
        if references_list:
            reference_links = references_list.find_all('a', class_='ssc-ext-link')
            references = [link['href'] for link in reference_links]
    else:
        references = f'Not found references for {cve_id}'

    return {
        'cve_id': cve_id,
        'published_date': published_date,
        'updated_date': updated_date,
        'cwe_id': cwe_id,
        'total_vuln_cat': total_vuln_cat,
        'base_score':base_score,
        'base_severity':base_severity,
        'vector':vector,
        'exploitability':exploitability,
        'impact':impact
    }

In [26]:
cve_details=get_cve_details('CVE-2024-0508')

DEBUGGING -- cve_id:CVE-2024-0508


C:\Program Files\Anaconda3\envs\pyth3_p1\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cvedetails.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ERROR CVE-2024-0508>> Failed to parse HTML elements cve_content_div << 

 


In [27]:
cve_details

{'cve_id': 'CVE-2024-0508'}

In [18]:
df = pd.read_excel(r'cves_list.xlsx', sheet_name='CVEs')
#print(df)
df.head(7)

,cve_id
0,CVE-2024-20709
1,CVE-2024-20721
2,CVE-2023-27859
3,CVE-2023-36025
4,CVE-2023-27524
5,CVE-2024-0508


In [8]:
#en una lista NO
#result = [get_cve_details(x) for x in df['cve_id']]

In [33]:

#FUNCIONA
cve_details=[get_cve_details(x) for x in df['cve_id']]
df = pd.DataFrame.from_dict(cve_details)
out=df.to_json(orient='records', lines=True)
with open('json_cves.txt', 'w') as f:
    f.write(out)

DEBUGGING -- cve_id:CVE-2024-20709


C:\Program Files\Anaconda3\envs\pyth3_p1\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cvedetails.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


DEBUGGING -- cve_id:CVE-2024-20721


C:\Program Files\Anaconda3\envs\pyth3_p1\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cvedetails.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


DEBUGGING -- cve_id:CVE-2023-27859


...

DEBUGGING -- cve_id:CVE-2023-27524


C:\Program Files\Anaconda3\envs\pyth3_p1\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cvedetails.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


DEBUGGING -- cve_id:CVE-2024-0508


C:\Program Files\Anaconda3\envs\pyth3_p1\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cvedetails.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ERROR CVE-2024-0508>> Failed to parse HTML elements cve_content_div << 

 


In [29]:
cve_details

[{'cve_id': 'CVE-2024-20709',
  'published_date': '2024-01-15 13:15:08',
  'updated_date': '2024-01-23 19:28:39',
  'cwe_id': 'CWE-20 Improper Input Validation',
  'total_vuln_cat': ['Input validation', 'Denial of service'],
  'base_score': '5.5',
  'base_severity': 'MEDIUM',
  'vector': 'CVSS:3.1/AV:L/AC:L/PR:N/UI:R/S:U/C:N/I:N/A:H',
  'exploitability': '1.8',
  'impact': '3.6'},
 {'cve_id': 'CVE-2024-20721',
  'published_date': '2024-01-15 13:15:08',
  'updated_date': '2024-01-23 19:28:30',
  'cwe_id': 'CWE-20 Improper Input Validation',
  'total_vuln_cat': ['Input validation', 'Denial of service'],
  'base_score': '5.5',
  'base_severity': 'MEDIUM',
  'vector': 'CVSS:3.1/AV:L/AC:L/PR:N/UI:R/S:U/C:N/I:N/A:H',
  'exploitability': '1.8',
  'impact': '3.6'},
 {'cve_id': 'CVE-2023-27859',
  'published_date': '2024-01-22 20:15:47',
  'updated_date': '2024-01-25 02:03:31',
  'cwe_id': 'Not found CWE ids for CVE-2023-27859',
  'total_vuln_cat': ['Execute code'],
  'base_score': '6.5',
  'ba

In [35]:

#FUNCIONA
df = pd.DataFrame.from_dict(cve_details)
df.to_excel("cves_output.xlsx", index=False)

           cve_id       published_date         updated_date  \
0  CVE-2024-20709  2024-01-15 13:15:08  2024-01-23 19:28:39   
1  CVE-2024-20721  2024-01-15 13:15:08  2024-01-23 19:28:30   
2  CVE-2023-27859  2024-01-22 20:15:47  2024-01-25 02:03:31   
3  CVE-2023-36025  2023-11-14 18:15:32  2023-11-21 01:33:14   
4  CVE-2023-27524  2023-04-24 16:15:08  2024-01-21 03:15:36   
5   CVE-2024-0508                  NaN                  NaN   

                                              cwe_id  \
0                   CWE-20 Improper Input Validation   
1                   CWE-20 Improper Input Validation   
2               Not found CWE ids for CVE-2023-27859   
3               Not found CWE ids for CVE-2023-36025   
4  CWE-1188 Initialization of a Resource with an ...   
5                                                NaN   

                          total_vuln_cat base_score base_severity  \
0  [Input validation, Denial of service]        5.5        MEDIUM   
1  [Input validation, Denia